In [1]:
import matplotlib.pyplot as plt
import monai
import numpy as np
import torch
from monai.data import DataLoader, Dataset
from pathlib import Path
from monai.transforms.utils import allow_missing_keys_mode
from monai.transforms import BatchInverseTransform
from monai.networks.nets import ResNet
import nibabel as nib
from tqdm import tqdm
import medpy.metric as metric
import os
import dxchange

astropy module not found


In [28]:
def getBugData(dataset_path: Path):
    dataset = []
    for idx, item in enumerate(os.listdir(dataset_path)):
        one_hot_v = np.zeros(12)
        one_hot_v[idx] = 1
        if item == "BC" or item == "BF" or item == "BL":
            for file in os.listdir(str(dataset_path) + "/"+ item):
                dataset.append({'image':str(dataset_path) + "/"+ item + "/" + file,
                                 "class": str(item),
                                 "label": one_hot_v})
    return dataset

DATA_PATH = "/dtu/3d-imaging-center/courses/02510/data/Bugs/bugnist_128/"
Files = [{"image": "/dtu/3d-imaging-center/courses/02510/groups/group_Anhinga/Linea/data_test/image2.nii.gz", "label": "test", "class": [0]}]

# 1. Data. Make a 70-10-20% train-validation-test split here
#Files = getBugData(dataset_path=Path(DATA_PATH))
print(Files[0]['image'])

#valFiles = getBugData(dataset_path=Path(DATA_PATH))  
#testFiles = getBugData(dataset_path=Path(DATA_PATH))

/dtu/3d-imaging-center/courses/02510/groups/group_Anhinga/Linea/data_test/image2.nii.gz


In [27]:
#load Files[0]['image']
import PIL.Image as Image
import numpy as np
import dxchange

img = dxchange.read_tiff(Files[0]['image'])
print(img.shape)

# save as nii
nii = nib.Nifti1Image(img, np.eye(4))
nib.save(nii, '/dtu/3d-imaging-center/courses/02510/groups/group_Anhinga/Linea/data_test/image2.nii.gz')

(128, 64, 64)


In [29]:
train_transforms = monai.transforms.Compose([
    monai.transforms.LoadImaged(keys='image'),
    monai.transforms.EnsureChannelFirstd(keys='image'),
])

BATCH_SIZE = 1
train_dataset = Dataset(data=Files, transform=train_transforms)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
#val_dataset = Dataset(data=valFiles, transform=val_transforms)
#val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)


In [30]:
for item in train_loader:
    print(item['image'].shape)

torch.Size([1, 1, 128, 64, 64])


In [31]:
model = ResNet(block = 'basic',
               layers = [3, 4, 6, 3],
                spatial_dims=3, 
                n_input_channels=1,
                num_classes = 3,
                block_inplanes = [16, 32, 64, 128],
)

loss_function = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), 1e-4)


In [32]:
loss = []

for epoch in range(10):
    model.train()
    epoch_loss = 0
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        image = batch['image'].float()
    
        label = batch['label'].long()
        output = model(image)
        loss = loss_function(output, label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(epoch, epoch_loss)
    loss.append(epoch_loss)

  0%|                                                 | 0/1 [00:00<?, ?it/s]


AttributeError: 'list' object has no attribute 'long'